In [3]:
import pandas as pd
import requests
import json
import time
from tqdm import tqdm
import numpy as np

# Ollama API 설정
OLLAMA_API_URL = "http://localhost:11434/api/generate"
MODEL_NAME = "jinbora/deepseek-r1-Bllossom:70B"

def translate_with_ollama(text, source_lang="classical_chinese", target_lang="korean"):
    """
    Ollama API를 사용하여 텍스트를 번역합니다.
    """
    if pd.isna(text) or text == "":
        return ""
    
    # 번역 프롬프트 구성
    if source_lang == "classical_chinese":
        prompt = f"""다음 한문을 현대 한국어로 번역해주세요. 정확하고 자연스러운 번역을 제공해주세요.

한문: {text}

한국어 번역:"""
    else:
        prompt = f"""다음 텍스트를 현대 한국어로 번역해주세요:

원문: {text}

번역:"""
    
    # API 요청 데이터
    payload = {
        "model": MODEL_NAME,
        "prompt": prompt,
        "stream": False,
        "options": {
            "temperature": 0.1,  # 낮은 temperature로 일관된 번역
            "num_predict": 500   # 최대 토큰 수
        }
    }
    
    try:
        response = requests.post(OLLAMA_API_URL, json=payload, timeout=120)
        response.raise_for_status()
        
        result = response.json()
        translation = result.get('response', '').strip()
        
        return translation
        
    except requests.exceptions.RequestException as e:
        print(f"API 요청 오류: {e}")
        return f"[번역 오류: {str(e)}]"
    except json.JSONDecodeError as e:
        print(f"JSON 파싱 오류: {e}")
        return "[번역 오류: JSON 파싱 실패]"
    except Exception as e:
        print(f"예상치 못한 오류: {e}")
        return f"[번역 오류: {str(e)}]"

# 데이터 로드
print("데이터 로딩 중...")
df = pd.read_csv('/home/work/songhune/korean_R-CoA/data/gwashi.csv', encoding='euc-kr')

print(f"총 {len(df)}개 행 로드됨")
print(f"abstract 컬럼 결측치: {df['abstract'].isnull().sum()}개")
print(f"contents 컬럼 결측치: {df['contents'].isnull().sum()}개")

데이터 로딩 중...
총 3348개 행 로드됨
abstract 컬럼 결측치: 2682개
contents 컬럼 결측치: 572개


In [ ]:
# 번역할 데이터 필터링 (결측치가 아닌 행만)
abstract_to_translate = df[df['abstract'].notna() & (df['abstract'] != '')].copy()
contents_to_translate = df[df['contents'].notna() & (df['contents'] != '')].copy()

print(f"번역할 abstract: {len(abstract_to_translate)}개")
print(f"번역할 contents: {len(contents_to_translate)}개")

# 새로운 컬럼 초기화
df['abstract_korean'] = ''
df['contents_korean'] = ''

In [ ]:
# Abstract 컬럼 번역
print("\nAbstract 컬럼 번역 시작...")
abstract_indices = abstract_to_translate.index.tolist()

for i, idx in enumerate(tqdm(abstract_indices, desc="Abstract 번역 중")):
    original_text = df.loc[idx, 'abstract']
    
    # 번역 수행
    translated = translate_with_ollama(original_text, source_lang="classical_chinese")
    df.loc[idx, 'abstract_korean'] = translated
    
    # 진행 상황 출력 (10개마다)
    if (i + 1) % 10 == 0:
        print(f"\n진행률: {i+1}/{len(abstract_indices)}")
        print(f"원문: {original_text[:50]}...")
        print(f"번역: {translated[:50]}...")
    
    # API 과부하 방지를 위한 대기
    time.sleep(1)

print("\nAbstract 번역 완료!")

In [ ]:
# Contents 컬럼 번역
print("\nContents 컬럼 번역 시작...")
contents_indices = contents_to_translate.index.tolist()

for i, idx in enumerate(tqdm(contents_indices, desc="Contents 번역 중")):
    original_text = df.loc[idx, 'contents']
    
    # 번역 수행
    translated = translate_with_ollama(original_text, source_lang="classical_chinese")
    df.loc[idx, 'contents_korean'] = translated
    
    # 진행 상황 출력 (10개마다)
    if (i + 1) % 10 == 0:
        print(f"\n진행률: {i+1}/{len(contents_indices)}")
        print(f"원문: {original_text[:50]}...")
        print(f"번역: {translated[:50]}...")
    
    # API 과부하 방지를 위한 대기
    time.sleep(1)

print("\nContents 번역 완료!")

In [ ]:
# 결과 저장
output_path = '/home/work/songhune/korean_R-CoA/data/gwashi_translated.csv'
df.to_csv(output_path, index=False, encoding='utf-8')
print(f"\n번역된 데이터가 저장되었습니다: {output_path}")

# 번역 결과 요약
translated_abstract_count = df[df['abstract_korean'] != ''].shape[0]
translated_contents_count = df[df['contents_korean'] != ''].shape[0]

print(f"\n번역 결과 요약:")
print(f"- Abstract 번역 완료: {translated_abstract_count}개")
print(f"- Contents 번역 완료: {translated_contents_count}개")
print(f"- 총 데이터 행 수: {len(df)}개")

In [ ]:
# 번역 결과 샘플 확인
print("\n번역 결과 샘플:")
print("="*80)

# Abstract 샘플
abstract_samples = df[df['abstract_korean'] != ''].head(3)
for i, row in abstract_samples.iterrows():
    print(f"\n[Abstract 샘플 {i+1}]")
    print(f"원문: {row['abstract']}")
    print(f"번역: {row['abstract_korean']}")
    print("-" * 50)

# Contents 샘플
contents_samples = df[df['contents_korean'] != ''].head(3)
for i, row in contents_samples.iterrows():
    print(f"\n[Contents 샘플 {i+1}]")
    print(f"원문: {row['contents'][:100]}...")
    print(f"번역: {row['contents_korean'][:100]}...")
    print("-" * 50)